# Методи ансамблювання моделей

In [1]:
!pip install mlxtend

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlxtend.plotting import plot_decision_regions

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Імпорт даних

In [3]:
df = pd.read_csv('diabetes.csv')

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

In [ ]:
X = df[['Glucose', 'Age']]
y = df['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [ ]:
def get_report(model, X_test, y_true):
    y_pred = model.predict(X_test)
    print(classification_report(y_true, y_pred, digits=4))

# kNN (k-найближчі сусіди)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)

In [ ]:
get_report(knn, X_test, y_test)

# Дерево Рішень

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
get_report(dt, X_test, y_test)

# Логістична регресія

In [ ]:
log_reg = LogisticRegression(solver='lbfgs', max_iter = 1000)

log_reg.fit(X_train, y_train)
get_report(log_reg, X_test, y_test)

# Ансамблювання: Голосування (Voting)

In [ ]:
from sklearn.ensemble import VotingClassifier

Для `VotingClassifier` нам не потрібно мати навчені моделі. Всередині, `VotingClassifier` створює копії моделей і навчає їх самостійно, а потім агрегує.

In [ ]:
dt = DecisionTreeClassifier()
knn_best = KNeighborsClassifier(n_neighbors=8)
log_reg = LogisticRegression(solver='lbfgs')

Модель не навчена, тому не може передбачати.

In [ ]:
# dt.predict(X_test)

In [ ]:
# оглошуємо словник моделей
estimators=[('knn', knn_best), ('dt', dt), ('log_reg', log_reg)]

# створюємо voting класифікатор
ensemble = VotingClassifier(estimators)

ensemble.fit(X_train, y_train)
get_report(ensemble, X_test, y_test)

In [ ]:
?VotingClassifier

Після навчання початкові моделі залишилися ненавченими, тому `dt` все ще не може передбачати.

In [ ]:
# dt.predict(X_test)

Але оцінювач (estimator) був навчений у рамках ансамблю. Отримати всі оцінювачі ансамблю можемо так:

In [ ]:
ensemble.estimators_

А перевірити, що вони навчені - так:

In [ ]:
ensemble.estimators_[1].predict(X_test)

М'яке голосування (soft voting) можливе лише тоді, коли всі ваші класифікатори можуть обчислювати ймовірності результатів (predict proba). М'яке голосування дає найкращий результат шляхом усереднення ймовірностей, обчислених окремими алгоритмами.  
Давайте спробуємо застосувати м'яке голосування.

In [ ]:
import inspect
from sklearn.tree import DecisionTreeRegressor

inspect.getmembers(DecisionTreeRegressor, predicate=inspect.isfunction)

In [ ]:
ensemble = VotingClassifier(estimators, voting='soft')

ensemble.fit(X_train, y_train)
get_report(ensemble, X_test, y_test)

Можемо передати ваги учасників голосування:

In [ ]:
estimators

In [ ]:
ensemble = VotingClassifier(estimators, voting ='hard', weights=[2, 1, 10])

ensemble.fit(X_train, y_train)
get_report(ensemble, X_test, y_test)

# Стекинг (Stacking)

stack_method : {'auto', 'predict_proba', 'decision_function', 'predict'},             default='auto'
    Методи, що викликаються для кожного базового оцінювача. Це може бути:

    * якщо 'auto', він спробує викликати для кожного оцінювача
      `'predict_proba'`, `'decision_function'` або `'predict'` у цьому
      порядку.
    * в іншому випадку, один з `'predict_proba'`, `'decision_function'` або
      `'predict'`. Якщо метод не реалізований оцінювачем, буде викликана помилка.

In [ ]:
from sklearn.ensemble import StackingClassifier


estimators=[('knn', knn_best), ('dt', dt), ('log_reg', log_reg)]
stacking_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
stacking_clf.fit(X_train, y_train)
get_report(stacking_clf, X_test, y_test)

Ось атрибути стекінгу, до яких ви можете звернутися:

In [ ]:
vars(stacking_clf)

Можна подивитися, які значення видає фінальна модель при різних вхідних значеннях:

In [ ]:
X_train.head()

In [ ]:
stacking_clf.final_estimator_.predict_proba([[0.1,0.4,0.9]])

In [ ]:
stacking_clf.final_estimator_.predict([[0,1,1]])

Виходить, що насправді найбільш значущою моделлю є knn при прийнятті рішень.

# Беггінг (Bagging) та Пастинг (Pasting)

Реалізація `BaggingClassifier` в `sklearn` включає в себе кілька алгоритмів з літератури.

-  Якщо випадкові підмножини даних обираються без заміни, то цей метод називається Pasting.

- Якщо зразки взяті з заміною, то метод відомий як Bagging.

- Коли випадкові підмножини набору даних вибираються як випадкові підмножини ознак (features), тоді метод відомий як Random Subspaces.

- Коли побудовані базові оцінювачі на підмножинах як зразків, так і ознак, тоді метод відомий як Random Patches.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=4),
    n_estimators=1000,
    max_samples=500,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)
bag_clf.fit(X_train, y_train)

get_report(bag_clf, X_test, y_test)

## bootstrap=False - для алгоритму Pasting

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    max_samples=200,
    bootstrap=False,
    n_jobs=-1,
    random_state=42
)
bag_clf.fit(X_train, y_train)

get_report(bag_clf, X_test, y_test)

Параметр `oob_score : bool`, за замовчуванням `False` і означає `Чи використовувати поза out-of-bag зразки для оцінки помилки узагальнення.`

In [ ]:
bag_clf.oob_score

## Оцінка Out-of-Bag

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42,
    oob_score=True
)
bag_clf.fit(X_train, y_train)

get_report(bag_clf, X_test, y_test)

In [ ]:
bag_clf.oob_score_

# Випадковий ліс (Random Forest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=5, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)

get_report(rnd_clf, X_test, y_test)

In [ ]:
plot_decision_regions(X_train.values, y_train.values, rnd_clf);

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=1, max_leaf_nodes=8, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
plot_decision_regions(X_train.values, y_train.values, rnd_clf);

Підберемо гіперпараметри:

In [ ]:
rf = RandomForestClassifier(random_state=42)
params_rf = {
    'n_estimators': [1, 5, 10, 50, 100, 200],
    'max_leaf_nodes':[4,8,16,32]
}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(X_train, y_train)
rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)

get_report(rf_best, X_test, y_test)

Scoring - краще встановлювати свій, тому що інакше буде використовуватися той, що за замовчуванням у алгоритму навчання, а у RandomForestClassifier - accuracy (точність).

In [ ]:
rf = RandomForestClassifier(random_state=42)
params_rf = {'n_estimators': [1, 5, 10, 50, 100, 200],
            'max_leaf_nodes':[4, 8, 16, 32]}
rf_gs = GridSearchCV(rf, params_rf, cv=5, scoring='f1_micro')
rf_gs.fit(X_train, y_train)
rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)

get_report(rf_best, X_test, y_test)

In [ ]:
plot_decision_regions(X_train.values, y_train.values, rf_best);

# Інші цікаві способи використання Random Forest
## Регресія за допомогою випадкового лісу (Random forest regression)

In [ ]:
rng = np.random.RandomState(42)
x = 10 * rng.rand(200)

In [ ]:
def model(x, sigma=0.3):
    fast_oscillation = np.sin(5 * x)
    slow_oscillation = np.sin(0.5 * x)
    noise = sigma * rng.randn(len(x))

    return slow_oscillation + fast_oscillation + noise

In [ ]:
y = model(x)
plt.errorbar(x, y, 0.3, fmt='o');

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=200)
forest.fit(x[:, None], y)

In [ ]:
xfit = np.linspace(0, 10, 1000)
yfit = forest.predict(xfit[:, None])
ytrue = model(xfit, sigma=0)

In [ ]:
plt.figure(figsize=(10,6))
plt.errorbar(x, y, 0.3, fmt='o', alpha=0.5)
plt.plot(xfit, yfit, '-r');
plt.plot(xfit, ytrue, '-k', alpha=0.5);

## Random forest для класифікації зображень з цифрами

Спробуємо попрацювати з картинками 🖼
Для цього завантажимо набір рукописних цифр.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()

In [ ]:
digits.data.max()

Візуалізуємо наші дані.

In [ ]:
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')

    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

In [ ]:
digits.data.shape

In [ ]:
digits.data

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=0)
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(ytest, ypred)
plt.figure(figsize=(10,6))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=True)
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
digits.target

In [ ]:
err_df = pd.DataFrame(digits.target, columns=['true'])
ypred_full = model.predict(digits.data)

err_df['pred'] = ypred_full
err_df.head()

In [ ]:
err_df[err_df['true'] != err_df['pred']]

In [ ]:
err_df[:5]

In [ ]:
plt.imshow(digits.images[1]);